**Questions:**
- Should flow go
    - is_band $\to$ nb_albums, or
    - is_band $\to$ unknown_gender $\to$ nb_albums?
- 

In [ ]:
import pandas as pd
import numpy as np
import load_data as ld
import sankey as sk

import importlib
importlib.reload(sk)
importlib.reload(ld)

In [ ]:
# try:
#     display(LoadData.albums_data.head(2))
# except AttributeError:
#     LoadData.load_albums_data()
#     display(LoadData.albums_data.head(2))

# try:
#     display(LoadData.artists_data.head(2))
# except AttributeError:
#     LoadData.load_artists_data()
#     display(LoadData.artists_data.head(2))

# try:
#     display(LoadData.songs_data.head(2))
# except AttributeError:
#     LoadData.load_songs_data()
#     display(LoadData.songs_data.head(2))


# Type $\to$ gender

In [ ]:
sankey_diag = sk.Sankey()
type_gender = sk.TypeGender()
display(type_gender.df)
display(sankey_diag.sankey_df)

# Gender $\to$ nb_albums

In [ ]:
ld.LoadData.load_albums_data()
ld.LoadData.load_artists_data()
display(ld.LoadData.albums_data.head(2), ld.LoadData.artists_data.head(2))


In [ ]:
df_albums = (albums_data
    .copy()
    .groupby("id_artist").count()
    .rename(columns={"_id": "nb_albums"})
    .reset_index())
    
df_albums["id_artist"] = [element if element.startswith(
    "ObjectId(") else "ObjectId("+element+")" for element in df_albums["id_artist"]]

df_albums


In [ ]:
df_artists = (artists_data
    .copy()
    .rename(columns={"_id":"id_artist"})
    [["id_artist", "gender"]])

df_artists

In [ ]:
l1 = np.array(df_albums.index)
l2 = np.array(df_artists.index)
set(l1).symmetric_difference(set(l2))

In [ ]:
df_artists

In [ ]:
df = df_artists.copy()
df = df.merge(df_albums, on="id_artist", how="outer")
df = df.fillna({col: "unknown_"+col for col in df.columns})
df = df.replace({col: {"Other": "unknown_"+col for _ in df.columns} for col in df.columns})
df = df.drop(columns="id_artist")
df = df.value_counts()
df = df.reset_index()
df.columns = sankey_col
df

In [ ]:
def make_bins_nb_albums(nb_albums):
    bin_max = 10
    if nb_albums < 10:
        return int(nb_albums)
    else:
        return bin_max


def name_album_bins(nb_albums):
    bin_max = 10
    if nb_albums <= 1:
        return f"{nb_albums} album"
    elif nb_albums < 10:
        return f"{nb_albums} albums"
    else:
        return f"{bin_max}+ albums"


# Make bins
# df["target"] = df["target"].apply(lambda x: np.round(x, decimals=-1))
df["target"] = df["target"].apply(make_bins_nb_albums)
df = df.groupby(["source", "target"])
df = df.sum()
df = df.reset_index()
df = df.sort_values("target")
df["target"] = df["target"].apply(name_album_bins)
# df


In [ ]:
(np.ceil(0/5)*5).astype(int)-1

In [ ]:
sankey_df = sankey_df.append(df)
sankey_df.head(70)

In [ ]:
sankey_df.to_csv("sankey.csv", index=False)

# Nb albums $\to$ nb songs

In [ ]:
df_songs = (songs_data
      .copy()
      [["id_album"]])

# add ObjectIt to cells missing it
df_songs["id_album"] = [element if element.startswith(
    "ObjectId(") else "ObjectId("+element+")" for element in df_songs["id_album"]]
pd.DataFrame(df_songs.value_counts()).reset_index()
